In [1]:
import sys
from pathlib import Path

repo_root = Path.cwd().resolve().parents[1]  # …/collision_of_two_bodies
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))


In [2]:
from two_body import Config, set_global_seeds
from two_body.core.telemetry import setup_logger
from two_body.logic.controller import ContinuousOptimizationController
from two_body.presentation.visualization import Visualizer
from two_body.simulation.rebound_adapter import ReboundSim
import numpy as np


In [3]:
import logging
from IPython.display import display, Markdown

class NotebookHandler(logging.Handler):
    def __init__(self):
        super().__init__()
        self.lines = []

    def emit(self, record):
        msg = self.format(record)
        self.lines.append(msg)
        print(msg)  # aparece en la celda conforme avanza

handler = NotebookHandler()
handler.setFormatter(logging.Formatter("[%(asctime)s] %(levelname)s - %(message)s"))

logger = setup_logger(level="DEBUG")
logger.handlers.clear()          # quita otros handlers previos
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)


In [ ]:
case = {
    "t_end_short": 300.0,
    "t_end_long": 6000.0,
    "dt": 0.2,
    "integrator": "ias15",
    "r0": ((-1.0, 0.0, 0.0), (1.0, 0.0, 0.0), (0.0, 0.3, 0.0)),
    "v0": ((0.0, -0.6, 0.0), (0.0, 0.6, 0.0), (0.0, 0.0, 0.0)),
    "mass_bounds": (
        (0.45, 0.55),   # estrella 1
        (0.45, 0.55),   # estrella 2
        (0.0, 1e-4),    # tercer cuerpo casi nulo
    ),
    "G": 1.0,
    "pop_size": 150,
    "n_gen_step": 1,
    "crossover": 0.85,
    "mutation": 0.05,
    "elitism": 1,
    "seed": 1234,
    "max_epochs": 60,
    "top_k_long": 15,
    "stagnation_window": 8,
    "stagnation_tol": 1e-4,
    "local_radius": 0.02,
    "radius_decay": 0.9,
    "time_budget_s": 1800.0,
    "eval_budget": 60000,
    "artifacts_dir": "artifacts/sitnikov_opt",
    "save_plots": True,
    "headless": False,
}


In [5]:
cfg = Config(**case)
set_global_seeds(cfg.seed)
logger = setup_logger()

class SitnikovReboundSim(ReboundSim):
    def setup_simulation(self, *args, **kwargs):
        sim = super().setup_simulation(*args, **kwargs)
        def clamp(sim_obj):
            if len(sim_obj.particles) > 2:
                p = sim_obj.particles[2]
                p.x = 0.0
                p.vx = 0.0
        sim.additional_forces = clamp
        return sim

# Monkeypatch para que FitnessEvaluator use el adaptador sitnikov
from two_body.simulation import rebound_adapter
rebound_adapter.ReboundSim = SitnikovReboundSim


In [6]:
controller = ContinuousOptimizationController(cfg, logger=logger)
results = controller.run()
display(Markdown("### Logs capturados"))
display("\n".join(handler.lines))
results

[2025-10-21 17:55:13,558] WARNING - pymoo unavailable or setup failed: cannot import name 'SBXCrossover' from 'pymoo.operators.crossover.sbx' (c:\Users\emicr\anaconda3\envs\grav2body\Lib\site-packages\pymoo\operators\crossover\sbx.py). Using internal sampler.
[2025-10-21 17:55:13,559] INFO - Starting optimization | pop=80 | dims=3 | time_budget=1800.0s | eval_budget=6000


c:\Users\emicr\anaconda3\envs\grav2body\Lib\site-packages\rebound\__init__.py:58: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


[2025-10-21 17:55:17,757] INFO - Epoch 0 | new global best (short) λ≈ -0.010498 | masses=(0.532414, 0.475715, 1.4e-05)
[2025-10-21 17:55:34,443] INFO - Epoch 0 complete | λ_short≈ -0.010498 | evals short/long=80/16 | total evals=96 | radius=0.0200
[2025-10-21 17:55:52,000] INFO - Epoch 1 complete | λ_short≈ -0.010498 | evals short/long=80/16 | total evals=192 | radius=0.0200
[2025-10-21 17:55:56,126] INFO - Epoch 2 | new global best (short) λ≈ -0.010743 | masses=(0.544603, 0.487263, 4.4e-05)
[2025-10-21 17:56:09,846] INFO - Epoch 2 complete | λ_short≈ -0.010743 | evals short/long=80/16 | total evals=288 | radius=0.0200
[2025-10-21 17:56:27,558] INFO - Epoch 3 complete | λ_short≈ -0.010743 | evals short/long=80/16 | total evals=384 | radius=0.0200
[2025-10-21 17:56:45,280] INFO - Epoch 4 complete | λ_short≈ -0.010743 | evals short/long=80/16 | total evals=480 | radius=0.0200
[2025-10-21 17:57:03,584] INFO - Epoch 5 complete | λ_short≈ -0.010743 | evals short/long=80/16 | total evals=576

### Logs capturados

"[2025-10-21 17:55:13,558] WARNING - pymoo unavailable or setup failed: cannot import name 'SBXCrossover' from 'pymoo.operators.crossover.sbx' (c:\\Users\\emicr\\anaconda3\\envs\\grav2body\\Lib\\site-packages\\pymoo\\operators\\crossover\\sbx.py). Using internal sampler.\n[2025-10-21 17:55:13,559] INFO - Starting optimization | pop=80 | dims=3 | time_budget=1800.0s | eval_budget=6000\n[2025-10-21 17:55:17,757] INFO - Epoch 0 | new global best (short) λ≈ -0.010498 | masses=(0.532414, 0.475715, 1.4e-05)\n[2025-10-21 17:55:34,443] INFO - Epoch 0 complete | λ_short≈ -0.010498 | evals short/long=80/16 | total evals=96 | radius=0.0200\n[2025-10-21 17:55:52,000] INFO - Epoch 1 complete | λ_short≈ -0.010498 | evals short/long=80/16 | total evals=192 | radius=0.0200\n[2025-10-21 17:55:56,126] INFO - Epoch 2 | new global best (short) λ≈ -0.010743 | masses=(0.544603, 0.487263, 4.4e-05)\n[2025-10-21 17:56:09,846] INFO - Epoch 2 complete | λ_short≈ -0.010743 | evals short/long=80/16 | total evals=2

{'status': 'completed',
 'best': {'masses': [0.55, 0.4912081196325653, 0.0],
  'lambda': -0.012085034637521896,
  'fitness': 0.012085034637521896,
  'm1': 0.55,
  'm2': 0.4912081196325653,
  'm3': 0.0},
 'evals': 5760,
 'epochs': 60}

In [ ]:
from two_body.logic.fitness import FitnessEvaluator
from two_body.core.cache import HierarchicalCache

cache = HierarchicalCache()
evaluator = FitnessEvaluator(cache, cfg)

center = tuple((lo + hi) / 2.0 for lo, hi in cfg.mass_bounds)
baseline = evaluator.evaluate_batch([center], horizon="long")[0]
best = results["best"]["fitness"]

print(f"λ inicial = {-baseline:.6f}, λ óptimo = {-best:.6f}")


print(f"λ inicial = {-baseline:.6f}, λ óptimo = {-best:.6f}")

viz = Visualizer(headless=cfg.headless)
sim_builder = SitnikovReboundSim(G=cfg.G, integrator=cfg.integrator)
sim = sim_builder.setup_simulation(results["best"]["masses"], cfg.r0, cfg.v0)
traj = sim_builder.integrate(sim, t_end=cfg.t_end_long, dt=cfg.dt)
viz.quick_view([traj[:, 0, :3], traj[:, 1, :3], traj[:, 2, :3]])
